## Zusammenführen von Whisper-Transkript und Pyannote-Diarization (Matching)

In diesem Workshop lernst du, wie man das **transkribierte** Ergebnis von **Whisper** (Chunks mit Zeitstempeln und Text) mit dem Ergebnis der **Pyannote Speaker Diarization** (Sprecher und deren Zeitintervalle) kombiniert. Ziel ist es, jedem Text-Chunk einen Sprecher zuzuweisen.

Voraussetzungen:
- Eine Datei mit dem **Whisper**-Output, z. B. `whisper_data.json`, die zumindest folgende Struktur hat:
  ```json
  {
    "text": "...",
    "chunks": [
      {
        "timestamp": [ start, end ],
        "text": "..."
      },
      ...
    ]
  }
  ```

- Eine Datei mit den Pyannote-Ergebnissen, z. B. diarization_data.json, welche ein Array von Segmenten enthält:
```json
[
  {
    "start": 0.0,
    "end": 1.5,
    "speaker": "SPEAKER_00"
  },
  ...
]
```

### 1. Imports

In [25]:
import json

### 2. Dateipfade spezifizieren
- `whisper_folder` verweist auf den Ordner mit den Whisper-Ergebnissen.
- `pyannote_folder` verweist auf den Ordner mit den Pyannote-Ergebnissen.
- `matched_transcripts_folder` ist der Zielordner, in dem wir die **kombinierten** Daten ablegen.
- `audio_file_name` ist der Basename der entsprechenden Audiodatei (z. B. `"Sonic-Manus-Pitch"`).


In [26]:
# Spezifierung von Verzeichnissen und Dateinamen
whisper_folder = "./outputs/whisper/"
pyannote_folder = "./outputs/pyannote/"
matched_transcripts_folder = "./outputs/matched_transcripts/"
audio_file_name = "Sonic-Manus-Pitch"

### 3. Einlesen der JSON-Dateien
Wir lesen:
1. Die Whisper-Ergebnisse (`whisper_data.json`) mit den Chunks.
2. Die Pyannote-Ergebnisse (`diarization_data.json`) mit den Sprechersegmenten.

Anschließend liegen uns zwei Python-Objekte vor:
- `whisper_data` mit u. a. `whisper_data["chunks"]`
- `diarization_data` als Liste von Dictionaries mit `[{"start":..., "end":..., "speaker":...}, ...]`


In [27]:
# Dateien einlesen
with open(f"{whisper_folder}{audio_file_name}.json", "r") as whisper_file:
    whisper_data = json.load(whisper_file)

with open(f"{pyannote_folder}{audio_file_name}.json", "r") as diarization_file:
    diarization_data = json.load(diarization_file)

### 4. Zusammenführen der Chunks

Für jeden **Whisper-Chunk** (mit `start`, `end` und `text`) suchen wir den passenden **Sprecher** in den Diarization-Daten. 

**Logik**:
- Falls das Zeitintervall `[chunk_start, chunk_end]` vollständig in `[segment.start, segment.end]` liegt, ordnen wir diesen Sprecher zu.
- Falls es eine Überlappung gibt (nicht 100%ig passend, aber Start oder Ende überschneiden sich), weisen wir auch diesen Sprecher zu. 

Wenn kein Sprecher-Segment gefunden wird, setzen wir `speaker = "Unknown"` und geben eine Warnung aus.


Am Ende erstellen wir ein Array `merged_output`, in dem jeder Eintrag so aufgebaut ist:
```json
{
  "speaker": "SPEAKER_XX",
  "text": "...",
  "start": 0.0,
  "end": 1.2
}
```


In [28]:
# Liste für die zusammengeführten Ergebnisse
merged_output = []

# Durch alle Whisper-Chunks iterieren
for chunk in whisper_data["chunks"]:
    chunk_start = chunk["timestamp"][0]
    chunk_end = chunk["timestamp"][1]
    chunk_text = chunk["text"]

    # Standard-Sprecher = "Unknown"
    speaker = "Unknown"

    # Überprüfe, ob dieses Chunk mit einem Diarization-Segment übereinstimmt
    for segment in diarization_data:
        if chunk_start >= segment["start"] and chunk_end <= segment["end"]:
            # Das Whisper-Chunk liegt komplett in diesem Segment
            speaker = segment["speaker"]
            break
        elif chunk_start <= segment["end"] and chunk_end >= segment["start"]:
            # Teil-Überlappung
            speaker = segment["speaker"]
            break

    if speaker == "Unknown":
        print(f"Warning: No speaker found for chunk '{chunk_text}' [{chunk_start:.2f}s to {chunk_end:.2f}s]")

    merged_output.append({
        "speaker": speaker,
        "text": chunk_text.strip(),
        "start": chunk_start,
        "end": chunk_end
    })

### 5. Ausgabe von Matching

In [29]:
print(json.dumps(merged_output, indent=4))

[
    {
        "speaker": "SPEAKER_01",
        "text": "Moin, was ist denn das Sonic Manus Projekt?",
        "start": 0.0,
        "end": 3.2
    },
    {
        "speaker": "SPEAKER_01",
        "text": "Das Sonic Manus Projekt ist ein kairosiertes Transcription Tool.",
        "start": 3.2,
        "end": 7.04
    },
    {
        "speaker": "SPEAKER_00",
        "text": "Das Tolle daran ist, dass es auch die unterschiedlichen Sprecher in einem Gespr\u00e4ch aufgeteilt bekommt.",
        "start": 7.04,
        "end": 13.0
    },
    {
        "speaker": "SPEAKER_01",
        "text": "Okay, aber das kann Teams doch l\u00e4ngst.",
        "start": 13.0,
        "end": 15.88
    },
    {
        "speaker": "SPEAKER_01",
        "text": "Er schon, aber Teams k\u00f6nnen daf\u00fcr einfach die unterschiedlichen Ger\u00e4te von den Benutzern verwenden",
        "start": 15.88,
        "end": 21.0
    },
    {
        "speaker": "SPEAKER_01",
        "text": "und wir k\u00f6nnen das ganz

### 6. Datei Speichern

- Nun speichern wir das Ergebnis in einer `.json` in den Zielordner (`matched_transcripts_folder`).


In [30]:
# In Datei speichern
with open(f"{matched_transcripts_folder}{audio_file_name}.json", "w") as output_file:
    json.dump(merged_output, output_file, indent=4, ensure_ascii=False)

## Zusammenfassung

In diesem Schritt haben wir:
1. **Whisper**-Transkripte (Text + Zeitstempel) eingelesen.
2. **Pyannote**-Diarization-Segmente (Sprecher + Zeitintervalle) eingelesen.
3. Für jedes Whisper-Chunk basierend auf dem Zeitintervall einen **Sprecher** ermittelt.
4. Das Ergebnis in einer JSON-Datei gespeichert.

Damit hast du eine **Sprecher-annotierte Transkription**, in der jeder Abschnitt weiß, von wem er gesprochen wurde. Diese Daten können z. B. für **Interviews**, **Meetings** oder **Podcasts** sehr hilfreich sein, um mehr Informationen als nur Text zu haben.
